In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
import re
import io
import requests
import numpy as np
from datetime import datetime

In [41]:
# Load the Retail Store Sales dataset 
#step 1
url = "Building_Permits.csv"
df = pd.read_csv(url)

In [42]:
df.isnull().sum()

Permit Number                                 0
Permit Type                                   0
Permit Type Definition                        0
Permit Creation Date                          0
Block                                         0
Lot                                           0
Street Number                                 0
Street Number Suffix                      23366
Street Name                                   0
Street Suffix                               338
Unit                                      20177
Unit Suffix                               23406
Description                                   9
Current Status                                0
Current Status Date                           0
Filed Date                                    0
Issued Date                                   0
Completed Date                            14327
First Construction Document Date          23687
Structural Notification                   23080
Number of Existing Stories              

In [9]:
def clean_colname(name: str) -> str:
    """
    Normalize column names:
      - lower case
      - replace spaces and punctuation with underscores
      - remove repeated underscores
    """
    name = name.lower()
    name = re.sub(r"[^\w]+", "_", name)      # non-word -> underscore
    name = re.sub(r"_+", "_", name)          # collapse multiple underscores
    name = name.strip("_")
    return name


In [16]:
def to_numeric_safely(series: pd.Series) -> pd.Series:
    """
    Convert a pandas Series to numeric by stripping commas/currency symbols and coercing errors to NaN.
    Useful for cost/value columns that may contain "$", ",", or text like "N/A".
    """
    return pd.to_numeric(series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True), errors="coerce")

def mode_or_nan(series: pd.Series):
    """Return mode if exists else NaN"""
    try:
        m = series.mode(dropna=True)
        return m.iloc[0] if not m.empty else np.nan
    except Exception:
        return np.nan

In [12]:
print(f"Raw rows downloaded: {len(df)}")

Raw rows downloaded: 23789


In [17]:
# --- Step 2: Normalize column names -------------------------------------
df.columns = [clean_colname(c) for c in df.columns]
print("Columns after normalization:", list(df.columns)[:30])

Columns after normalization: ['permit_number', 'permit_type', 'permit_type_definition', 'permit_creation_date', 'block', 'lot', 'street_number', 'street_number_suffix', 'street_name', 'street_suffix', 'unit', 'unit_suffix', 'description', 'current_status', 'current_status_date', 'filed_date', 'issued_date', 'completed_date', 'first_construction_document_date', 'structural_notification', 'number_of_existing_stories', 'number_of_proposed_stories', 'voluntary_soft_story_retrofit', 'fire_only_permit', 'permit_expiration_date', 'estimated_cost', 'revised_cost', 'existing_use', 'existing_units', 'proposed_use']


In [19]:
# --- Step 3: Quick inspection & missingness report -----------------------
# Compute missingness percentages
missing_pct = df.isnull().mean().sort_values(ascending=False) * 100
missing_summary = pd.DataFrame({
    "column": missing_pct.index,
    "missing_percent": missing_pct.values,
    "non_null_count": df.shape[0] - df.isnull().sum().values
})



In [20]:
# Print top 15 most-missing columns for quick view
print("Top 15 columns by missing %:\n", missing_summary.head(15))

Top 15 columns by missing %:
                                     column  missing_percent  non_null_count
0                          tidf_compliance       100.000000           23789
1            voluntary_soft_story_retrofit       100.000000           23789
2         first_construction_document_date        99.571230           23789
3                              unit_suffix        98.390012           23789
4                     street_number_suffix        98.221867           23789
5                              site_permit        98.141998           23789
6                  structural_notification        97.019631           23789
7                         fire_only_permit        85.413426             423
8                                     unit        84.816512           23789
9                           completed_date        60.225314           23451
10                          existing_units        16.259616            3612
11                          proposed_units        15.66690

In [21]:
# --- Step 4: Parse common date columns (if present) -----------------------
# Common date-like columns in permits data: 'application_date', 'issue_date', 'complete_date', 'filing_date', etc.
date_cols_candidates = [c for c in df.columns if any(k in c for k in ("date", "time", "issued", "appl", "filed"))]
print("Date-like columns detected:", date_cols_candidates)

Date-like columns detected: ['permit_creation_date', 'current_status_date', 'filed_date', 'issued_date', 'completed_date', 'first_construction_document_date', 'permit_expiration_date']


In [22]:
def try_parse_dates(df, cols):
    for c in cols:
        if c in df.columns:
            # try a few common formats, coerce errors to NaT
            df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
    return df

df = try_parse_dates(df, date_cols_candidates)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6060\4284074536.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_6060\4284074536.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_6060\4284074536.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version 

In [23]:
# Create derived time features for the most important columns, if present
for col in ("application_date", "issue_date", "complete_date", "filing_date"):
    if col in df.columns:
        df[f"{col}_year"] = df[col].dt.year
        df[f"{col}_month"] = df[col].dt.month
        df[f"{col}_dayofweek"] = df[col].dt.dayofweek

# Example: compute time-to-issue if both dates exist
if "application_date" in df.columns and "issue_date" in df.columns:
    df["days_to_issue"] = (df["issue_date"] - df["application_date"]).dt.days

In [30]:
# --- Step 5: Handle location / geometry column ----------------------------
# Many DataSF datasets have a 'location' or 'geocoded_column' nested JSON; sometimes latitude/longitude provided.
# We'll attempt to extract common fields: latitude, longitude, location_address
loc_cols = [c for c in df.columns if "location" in c or "latitude" in c or "longitude" in c]
print("Location-ish columns found:", loc_cols)

# If there is a single 'location' column containing JSON with latitude/longitude, parse it.
if "location" in df.columns:
    # some rows might have JSON-like strings; try to parse common patterns
    def extract_lat(row):
        try:
            v = row
            if pd.isna(v): return np.nan
            # if looks like {"latitude":..., "longitude":...}
            if isinstance(v, str) and ("latitude" in v or "lon" in v or "coordinates" in v):
                # crude extraction of floating numbers
                nums = re.findall(r"-?\d+\.\d+", v)
                if len(nums) >= 2:
                    # assume lat, lon or lon,lat depending on format; choose heuristic: lat ~ between -90 and 90
                    a, b = float(nums[0]), float(nums[1])
                    if -90 <= a <= 90: return a
                    if -90 <= b <= 90: return b
            return np.nan
        except Exception:
            return np.nan
    def extract_lon(row):
        try:
            v = row
            if pd.isna(v): return np.nan
            if isinstance(v, str) and ("latitude" in v or "lon" in v or "coordinates" in v):
                nums = re.findall(r"-?\d+\.\d+", v)
                if len(nums) >= 2:
                    a, b = float(nums[0]), float(nums[1])
                    if -180 <= b <= 180: return b
                    if -180 <= a <= 180: return a
            return np.nan
        except Exception:
            return np.nan

    df["latitude"] = df["location"].apply(extract_lat)
    df["longitude"] = df["location"].apply(extract_lon)

Location-ish columns found: ['location']


In [26]:

# If lat/long exist in other columns, ensure numeric type
for c in ["latitude", "longitude"]:
    if c in df.columns:
        df[c] = to_numeric_safely(df[c])

In [27]:
# --- Step 6: Normalize numeric / currency fields --------------------------
# Frequently: 'estimated_cost', 'job_value', 'work_cost', or 'total_cost' appear as text with $ and commas
possible_money_cols = [c for c in df.columns if any(k in c for k in ("cost","value","fee","amt","estimate","estimated","valuation","job_value"))]
print("Possible money columns:", possible_money_cols)
for c in possible_money_cols:
    df[c] = to_numeric_safely(df[c])
    # create flag for originally-missing
    df[f"{c}_missing_flag"] = df[c].isna().astype(int)

Possible money columns: ['estimated_cost', 'revised_cost']


In [32]:
# --- Step 7: Clean and standardize key categorical columns ----------------
# Examples: 'permit_type', 'permit_status', 'permit_subtype', 'job_description', 'work_type', 'application_status'
cat_candidates = [c for c in df.columns if any(k in c for k in ("permit","status","type","job","work","description","use"))]
print("Categorical-like columns:", cat_candidates)

def clean_text_col(series: pd.Series) -> pd.Series:
    # standard cleaning: strip, lower, remove extra whitespace, normalize common tokens
    s = series.fillna("").astype(str).str.strip()
    s = s.str.replace(r"\s+", " ", regex=True)
    s = s.str.replace(r"[^0-9a-zA-Z\s\-\_\,\&\/\(\)]", "", regex=True)  # allow some punctuation
    s = s.str.lower()
    s = s.replace({"": np.nan})
    return s

for c in cat_candidates:
    df[c] = clean_text_col(df[c])
# Example: collapse long text 'job_description' into a shorter 'job_short' (first 80 chars)
if "job_description" in df.columns:
    df["job_short"] = df["job_description"].astype(str).str.slice(0, 120).str.strip()

Categorical-like columns: ['permit_number', 'permit_type', 'permit_type_definition', 'permit_creation_date', 'description', 'current_status', 'current_status_date', 'fire_only_permit', 'permit_expiration_date', 'existing_use', 'proposed_use', 'existing_construction_type', 'existing_construction_type_description', 'proposed_construction_type', 'proposed_construction_type_description', 'site_permit']


In [33]:

# --- Step 8: Handle missing values intelligently --------------------------
# Strategy used here:
#  - For numeric columns: impute with median + keep a missing-flag column (already created for money columns)
#  - For categorical columns: impute with mode and create missing-flag
#  - For address components: keep as-is but try to extract clean 'street' and 'zipcode' if possible

# Numeric imputation example
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# exclude automatically created flags and lat/long if you prefer; here we impute only real numeric columns (not flags)
numeric_to_impute = [c for c in numeric_cols if not c.endswith("_flag") and c not in ("latitude","longitude")]
for c in numeric_to_impute:
    med = df[c].median(skipna=True)
    df[f"{c}_missing_flag"] = df[c].isna().astype(int)
    df[c] = df[c].fillna(med)

# Categorical imputation example (mode)
categorical_cols = [c for c in df.columns if df[c].dtype == "object"]
categorical_to_impute = [c for c in categorical_cols if df[c].isnull().mean() > 0 and df[c].nunique(dropna=True) < 500]
for c in categorical_to_impute:
    mode_val = mode_or_nan(df[c])
    df[f"{c}_missing_flag"] = df[c].isna().astype(int)
    if pd.notna(mode_val):
        df[c] = df[c].fillna(mode_val)

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [34]:
# --- Step 9: Address parsing (simple heuristics) --------------------------
# If there is an 'address' column, attempt to extract zip code and street number
addr_col = None
for candidate in ("job_address", "address", "full_address", "street_address"):
    if candidate in df.columns:
        addr_col = candidate
        break

if addr_col:
    # extract zipcode: last 5-digit group in the string
    df["zipcode"] = df[addr_col].astype(str).str.extract(r"(\d{5})(?:-?\d{0,4})?$", expand=False)
    # extract street number (first number group)
    df["street_number"] = df[addr_col].astype(str).str.extract(r"^\s*(\d+)\s+", expand=False)
    # street name (very naive: remove leading number and trailing zip)
    df["street_name"] = df[addr_col].astype(str).str.replace(r"^\s*\d+\s+", "", regex=True).str.replace(r"\s+\d{5}$", "", regex=True).str.strip()
    # clean blanks to NaN
    df["zipcode"] = df["zipcode"].replace({"": np.nan})
    df["street_number"] = df["street_number"].replace({"": np.nan})
    df["street_name"] = df["street_name"].replace({"": np.nan})

In [35]:
# --- Step 10: Remove duplicates & low-information columns -----------------
pre_dupe = len(df)
df = df.drop_duplicates()
post_dupe = len(df)
print(f"Removed {pre_dupe - post_dupe} duplicate rows.")

# Drop columns with > 95% missing (low information)
high_missing_cols = missing_summary[missing_summary["missing_percent"] > 95]["column"].tolist()
# only drop if they still exist in df
cols_to_drop = [c for c in high_missing_cols if c in df.columns]
print("Dropping extremely sparse columns:", cols_to_drop)
df = df.drop(columns=cols_to_drop, errors="ignore")

# --- Step 11: Cast final datatypes for compactness ------------------------
# Convert small-int-year columns to Int64 (nullable integer) where appropriate
for c in df.columns:
    if re.search(r"_year$|_month$|_dayofweek$|_flag$", c) and c in df.columns:
        try:
            df[c] = df[c].astype("Int64")
        except Exception:
            pass


Removed 0 duplicate rows.
Dropping extremely sparse columns: ['tidf_compliance', 'voluntary_soft_story_retrofit', 'first_construction_document_date', 'unit_suffix', 'street_number_suffix', 'site_permit', 'structural_notification']


In [36]:
# --- Step 12: Final summary & save --------------------------------------
summary = {
    "rows_final": len(df),
    "columns_final": df.shape[1],
    "sample_columns": list(df.columns)[:40]
}
print("Final summary:", summary)



Final summary: {'rows_final': 23789, 'columns_final': 65, 'sample_columns': ['permit_number', 'permit_type', 'permit_type_definition', 'permit_creation_date', 'block', 'lot', 'street_number', 'street_name', 'street_suffix', 'unit', 'description', 'current_status', 'current_status_date', 'filed_date', 'issued_date', 'completed_date', 'number_of_existing_stories', 'number_of_proposed_stories', 'fire_only_permit', 'permit_expiration_date', 'estimated_cost', 'revised_cost', 'existing_use', 'existing_units', 'proposed_use', 'proposed_units', 'plansets', 'existing_construction_type', 'existing_construction_type_description', 'proposed_construction_type', 'proposed_construction_type_description', 'supervisor_district', 'neighborhoods_analysis_boundaries', 'zipcode', 'location', 'record_id', 'estimated_cost_missing_flag', 'revised_cost_missing_flag', 'latitude', 'longitude']}


In [37]:
# Save cleaned dataframe
OUTPUT_CLEAN_CSV='output_clean_csv.csv'
MISSING_REPORT = "sf_missing_report.csv"
df.to_csv(OUTPUT_CLEAN_CSV, index=False)
print(f"Cleaned dataset saved to {OUTPUT_CLEAN_CSV}")
print("Missingness report was saved to", MISSING_REPORT)

Cleaned dataset saved to output_clean_csv.csv
Missingness report was saved to sf_missing_report.csv


In [40]:
df.isnull().sum()

permit_number                                          0
permit_type                                            0
permit_type_definition                                 0
permit_creation_date                                   0
block                                                  0
                                                      ..
existing_construction_type_description_missing_flag    0
proposed_construction_type_missing_flag                0
proposed_construction_type_description_missing_flag    0
site_permit_missing_flag                               0
neighborhoods_analysis_boundaries_missing_flag         0
Length: 65, dtype: int64